In [1]:
from typing import List, Tuple

from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

import numpy as np
import torch

import matplotlib.pyplot as plt

In [2]:
def dataset_select_split(dataset, threshold, test_size)\
        -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    # Select
    freq = np.bincount(dataset.target)
    cond = freq[dataset.target] >= threshold
    dataset.images = dataset.images[cond]
    dataset.target = dataset.target[cond]

    # Reclassify
    subst = {}
    target_reclass = []
    last_class = 0
    for actual_target in dataset.target:
        if actual_target in subst:
            target_reclass.append(subst[actual_target])
        else:
            subst[actual_target] = last_class
            target_reclass.append(last_class)
            last_class += 1
    dataset.target = np.array(target_reclass)

    train_data, test_data, train_labels, test_labels = train_test_split(
        dataset.images, dataset.target, test_size=test_size
    )

    # Convert to torch
    train_data = torch.unsqueeze(torch.from_numpy(train_data), dim=1)
    train_labels = torch.from_numpy(train_labels.astype(int))

    test_data = torch.unsqueeze(torch.from_numpy(test_data), dim=1)
    test_labels = torch.from_numpy(test_labels.astype(int))

    print("Dataset information:")
    print(f"    Classes: {last_class}")
    print(f"    Samples: {dataset.target.shape[0]}")

    return train_data, train_labels, test_data, test_labels

### Dataset Containers

In [3]:
class LFWDataset(torch.utils.data.Dataset):
    data: torch.Tensor
    labels: torch.Tensor

    def __init__(self, data: torch.Tensor, labels: torch.Tensor):
        self.data = data
        self.labels = labels

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()
        return self.data[item], self.labels[item]


### CNN Model

In [4]:
class Net(torch.nn.Module):
    def __init__(self, h1: int, h2: int, h3: int, h4: int, h5: int, h6: int):
        super(Net, self).__init__()
        self.__layers1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,  h1, kernel_size=5, stride=1),  # (1,  62, 47) -> (h1, 58, 43)
            torch.nn.Conv2d(h1, h2, kernel_size=5, stride=1),  # (h1, 58, 43) -> (h2, 54, 39)
            torch.nn.MaxPool2d(kernel_size=2, stride=1),       # (h2, 54, 39) -> (h2, 53, 38)
            torch.nn.Dropout(p=0.25)                           # (h2, 53, 38) -> (h2, 53, 38)
        )
        self.__layers2 = torch.nn.Sequential(
            torch.nn.Conv2d(h2, h3, kernel_size=3, stride=1),  # (h2, 53, 38) -> (h3, 51, 36)
            torch.nn.Conv2d(h3, h4, kernel_size=3, stride=1),  # (h3, 51, 36) -> (h4, 49, 34)
            torch.nn.MaxPool2d(kernel_size=3, stride=2),       # (h4, 49, 34) -> (h4, 24, 16)
            torch.nn.Dropout(p=0.25),                          # (h4, 24, 16) -> (h4, 24, 16)
        )
        self.__layers3 = torch.nn.Sequential(
            torch.nn.Linear(h4 * 24 * 16, h5),                 # (h4 * 24 * 16) -> (h5)
            torch.nn.Dropout(p=0.4),                           # (h5) -> (h5)
            torch.nn.Linear(h5, h6),                           # (h5) -> (h6)
            torch.nn.Dropout(p=0.4),                           # (h5) -> (h6)
            torch.nn.Softmax(dim=0)                            # (h5) -> (h6)
        )
        print("CNN Information:")
        print(f"    Hidden Layers:")
        print(f"        -> {h1} {h2} | {h3} {h4} | {h4 * 24 * 16} {h5} {h6} ->")
    
    def forward(self, x):
        x = self.__layers1(x)
        x = self.__layers2(x)
        x = x.view(x.size(0), -1)
        x = self.__layers3(x)
        return x

In [ ]:
def top_k_accuracy(k, target, output):
    batch_size = target.size[0]
    _, pred = output.topk(k, 1, True, True)
    
    pred = pred.t()
    correct = pred.eq(target.to(device).view(1, -1).expand_as(pred))
    
    correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
    currect_k.mul(100.0 / batch_size)
    
    return correct_k

In [25]:
def train_test(
        cnn: Net, optimizer: torch.optim.Adam, loss_func: torch.nn.CrossEntropyLoss,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader, epochs: int
) -> Tuple[List[float], List[float], List[float]]:

    train_losses: List[float] = []
    test_losses: List[float] = []
    accuracies: List[float] = []
    
    for epoch in range(epochs):
        cnn.train()

        train_loss = 0
        for step, (data, labels) in enumerate(train_loader):
            if torch.cuda.is_available():
                data = data.cuda()
                labels = labels.cuda()
            
            optimizer.zero_grad()
            logps = cnn.forward(data).long()
            loss = loss_func(logps, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        test_loss = 0
        accuracy = 0
        cnn.eval()
        with torch.no_grad():
            for data, labels in test_loader:
                if torch.cuda.is_available():
                    data = data.cuda()
                    labels = labels.cuda()
                
                logps = cnn.forward(data).long()
                loss = loss_func(logps, labels)
                test_loss += loss.item()

                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        train_loss /= len(test_loader)
        accuracy /= len(test_loader)

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        accuracies.append(accuracy)

        print(f"Epoch {epoch + 1:3d} / {epochs:3d}"
              f" Train loss: {train_loss:7.3f}"
              f" Test Loss: {test_loss:7.3f}"
              f" Test accuracy: {accuracy:7.3f}")
        cnn.train()

    return train_losses, test_losses, accuracies

In [26]:
def plot_loss(epochs: int, train_loss: List[float], test_loss: List[float]):
    plt.figure(figsize=(7, 5))
    plt.rcdefaults()
    plt.rc('font', size=14)
    
    plt.plot(range(1, epochs + 1), train_loss, ".-b", label="Training Loss")
    plt.plot(range(1, epochs + 1), test_loss, ".-r", label="Test Loss")
    plt.title("Loss Curves")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=3, fancybox=True, shadow=True)
    plt.show()


def plot_accuracies(epochs: int, accuracies: List[float]):
    plt.figure(figsize=(7, 5))
    plt.rcdefaults()
    plt.rc('font', size=14)
    
    plt.plot(range(1, epochs + 1), accuracies, ".-g", label="Accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylim(0, 1)
    # ax.legend(loc="upper right")
    plt.show()

#### Setup

In [27]:
epochs = 2000
batch_size = 50
threshold = 20
test_size = 0.2
learning_rate = 0.01
hidden_layers = [4, 16, 16, 10, 1900, 62]

dataset = fetch_lfw_people("/home/vlad/Facultate/Sem2/IA/Teme/Tema3/scikit_learn_data")
train_data, train_labels, test_data, test_labels = dataset_select_split(
    dataset, threshold, test_size
)
train_dataset = LFWDataset(train_data, train_labels)
test_dataset = LFWDataset(test_data, test_labels)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

print("Train dataset:")
print(f"    Data size: {train_dataset.data.size()}")
print(f"    Labels size: {train_dataset.labels.size()}")

print("Test dataset:")
print(f"    Data size: {test_dataset.data.size()}")
print(f"    Labels size: {test_dataset.labels.size()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn = Net(*hidden_layers)
cnn.train()
cnn = cnn.to(device)
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    print("Is Cuda!")
    cnn = cnn.cuda()
    loss_func = loss_func.cuda()

Dataset information:
    Classes: 62
    Samples: 3023
Train dataset:
    Data size: torch.Size([2418, 1, 62, 47])
    Labels size: torch.Size([2418])
Test dataset:
    Data size: torch.Size([605, 1, 62, 47])
    Labels size: torch.Size([605])
CNN Information:
    Hidden Layers:
        -> 4 16 | 16 10 | 3840 1900 62 ->
Is Cuda!


In [28]:
import pickle

do_train_test = True
if do_train_test:
    train_losses, test_losses, accuracies = train_test(
        cnn, optimizer, loss_func, train_loader, test_loader, epochs
    )
    with open("results_2000.pkl", "wb") as f:
        pickle.dump((train_losses, test_losses, accuracies), f)
else:
    with open("results_2000.pkl", "rb") as f:
        train_losses, test_losses, accuracies = pickle.load(f)

RuntimeError: "host_softmax" not implemented for 'Long'

In [ ]:
plot_accuracies(epochs, accuracies)
plot_loss(epochs, train_losses, test_losses)

In [ ]:
torch.save(cnn, "model.pth")

In [ ]:
cnn = torch.load("model.pth")
cnn.eval()

In [ ]:
from facenet_pytorch import InceptionResnetV1

# For a model pretrained on VGGFace2
model = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
from facenet_pytorch import MTCNN

mtcnn = MTCNN()

In [ ]:
mtcnn.detect(dataset.images[0])

In [ ]:
# https://github.com/davidsandberg/facenet/issues/1095